## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-01-08-50-00 +0000,bbc,Car loan scandal payouts row - what's it about?,https://www.bbc.com/news/articles/c2l9vvj097lo...
1,2025-08-01-08-46-44 +0000,nyt,Here is the latest.,https://www.nytimes.com/live/2025/07/31/busine...
2,2025-08-01-08-40-25 +0000,wapo,"Looting, chaos and Israeli gunfire prevent aid...",https://www.washingtonpost.com/world/2025/08/0...
3,2025-08-01-08-40-00 +0000,wsj,Stock Market Today: Stocks Fall on Trump's Tar...,https://www.wsj.com/livecoverage/jobs-report-t...
4,2025-08-01-08-37-31 +0000,nypost,Arkansas hairstylist reveals ‘eerie’ encounter...,https://nypost.com/2025/08/01/us-news/devils-d...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,78
49,tariffs,24
21,tariff,21
54,trade,16
408,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...,184
62,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...,152
241,2025-07-31-16-29-38 +0000,latimes,New Trump tariffs on Mexico delayed 90 days as...,https://www.latimes.com/world-nation/story/202...,149
88,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...,149
6,2025-08-01-08-30-00 +0000,wsj,President Trump promised Africa that trade wou...,https://www.wsj.com/world/africa/lesotho-tarif...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,184,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...
88,71,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...
100,65,2025-07-31-23-44-33 +0000,nypost,White House releases new tariff rates for doze...,https://nypost.com/2025/07/31/us-news/white-ho...
25,48,2025-08-01-05-44-24 +0000,nypost,Kamala Harris complains ‘the system’ is ‘broke...,https://nypost.com/2025/08/01/us-news/kamala-h...
260,47,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...
62,45,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...
81,45,2025-08-01-01-15-00 +0000,wsj,U.S. Envoy Steve Witkoff to Visit Gaza as Trum...,https://www.wsj.com/world/middle-east/u-s-envo...
282,41,2025-07-31-13-58-00 +0000,wsj,South Africa’s central bank cut its main repo ...,https://www.wsj.com/articles/south-african-res...
68,39,2025-08-01-02-17-25 +0000,nypost,Russian woman beaten in viral Cincinnati brawl...,https://nypost.com/2025/07/31/us-news/russian-...
82,34,2025-08-01-01-10-11 +0000,bbc,Three top officials were asleep or out of town...,https://www.bbc.com/news/articles/czernwez4epo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
